In [1]:
#!pip install pandas
#!pip install gensim
#!pip install mgzip
import pandas as pd
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec, KeyedVectors
from gensim import downloader
import gzip 
import math
import itertools
from time import time
import os
import tqdm as tq
import mgzip
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import logging

# Set up loss logging

In [2]:
class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        logging.debug('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        log_fil.write('Loss after epoch {}: {}\n\n'.format(self.epoch, loss_now))
        losses[filname][self.epoch] = loss_now
        self.epoch += 1

# Ngram generator from gzip files

In [6]:
class ngram_extractor_lite():
    def __init__(self, filname, blank=False, limit=None):
        self.limit = limit
        self.filname = filname
        self.blank = blank
    def __iter__(self):
        start = time()
        print("n", self.filname)
        with gzip.open(self.filname, "rt", encoding="ISO-8859-1") as fil: #just changed from r to rt so that it reads the csv separations
        # with gensim.utils.smart_open(os.path.join(self.dir_path, filname)) as fil:
            j = 0
            for line in tq.tqdm(itertools.islice(fil, self.limit)):
                #j+=1
                #if j > self.blank:
                 #   break
                line = gensim.utils.to_unicode(line).split("\t")
                ngram = line[0].split()
                if len(ngram) < 3:
                    continue
                try:
                    match_count = int(line[-1])
                except:
                    continue
                #prcssd_ngram = [word.split("_")[0] for word in ngram.lower().split()] #Get rid of POS tagging on end of words
                for i in range(match_count):
                    yield ngram
            t = time()-start
            print(f"Time taken: {t // 3600}H, {round((t % 3600)/60, 2)}M")

In [ ]:
def file2sent(fil):
    f = open(fil, "r")
    sents = []
    for line in tq.tqdm(itertools.islice(f)):
        line = gensim.utils.to_unicode(line)
        
    return sents
os.chdir("D:/google_ngrams/processed_data/business_text")
f = open(os.listdir()[0], "r").readlines()
sents = []
i = 0
for line in tq.tqdm(f):
    line = gensim.utils.to_unicode(line)
    print(line[:150])
    i +=1
    if i > 10:
        break
print(os.listdir()[0])
    

# Find where in the files the blanks (Pure POS tags) begin

In [ ]:
def find_blanks(dir):
    ''' Create two dictionaries, one containing the index of the first blank line in each file,
    and the second containing the length of the files'''
    if dir[-1] != "/":
        print("Add slash to end of dir.")
        return None
    filnames = [f"19{i}0.gz" for i in range(0,10)]
    infos = {"blank_ind": {}, "length": {}}
    os.chdir(dir)
    try:
        infos = pd.read_csv(dir + "file_infos.csv")
        infos = infos.set_index("filename")
        print("Infos loaded.")
        return infos
    except Exception as e:
        #print(e)
        ("Calculating infos...")
        for fil in filnames[:1]:
            print(fil)
            with gzip.open(fil, "rt", encoding = 'utf-8') as f:
                lst = []
                for line in f:
                    spl = line.split()
                    lst.append(spl)
            for i,spl in enumerate(lst[::-1]):
                #print("spl", spl, "eln", len(spl))
                if len(spl) > 2:
                    infos["blank_ind"][fil] = len(lst)-(i+1)
                    infos["length"][fil] = len(lst)
                    break
        infos = pd.DataFrame(infos)#.to_csv(dir + "file_infos")
        infos.index.name="filename"
        print(infos)
        infos.to_csv(dir + "file_infos.csv")
    return infos
infos = find_blanks("D:/google_ngrams/processed_data/gb_12_full_full/")
#TO ADD: ngram counts, averages, etc - everythign in expoloratory file

In [ ]:
#infos = infos.set_index("filename")
infos["blank_ind"]["1900.gz"]

# Iterable version (no loss calculation)

In [ ]:
assert gensim.models.word2vec.FAST_VERSION > -1 # check that it is the latest version of Word2vecngrams = ngram_extractor(dir_path, save_path, start_yr, end_yr, limit)

#an iterable version, so each iteration is ssaved (perhaps for each file there should be a save?):
epochs = 5
vocab = False
completed = 0
method = 1 # 1  sg, cbow = 0

size = 300 # vector size
min_count = 25
filnames = [f"19{i}0.gz" for i in range(0,10)]

os.chdir("D:/google_ngrams/processed_data/gb_12_full_full")

for filname in filnames[:1]:
    ngrams = ngram_extractor_lite(filname=filname)#, blank=infos["blank_id"][filname])
    for i in range(epochs-completed):
        if vocab:
            model = Word2Vec.load('w2vmodel_ng5_' + filname  + "_" + str("vocab"))
            print("Vocab loaded.")
        elif completed != 0:
            model = Word2Vec.load('w2vmodel_ng5_' + filname + "_" + str(completed))
            print("Model loaded.")
        elif i == 0:
            model = gensim.models.word2vec.Word2Vec(sg=method, vector_size=size, window=5, sg=1 min_count=25, workers=10, hs=0, negative=8, epochs=1, sample=10000, compute_loss=True, callbacks = [callback()])
            model.build_vocab(ngrams)
            print("Vocab built.")
            model.save('w2vmodel_ng5_'+ filname +'_' + str("vocab"))
        model.train(ngrams, total_examples = model.corpus_count, epochs = 1)
        model.save('w2vmodel_ng5_'+ filname +'_' + str(completed+i+1))
        print("Epoch " + str(completed + i + 1) + " completed.")


# Standard version - with loss calculation

In [ ]:
os.chdir("D:/google_ngrams/processed_data/business_text")
output_dir = "D:/google_ngrams/Vectors/business_vectors"

#filnames = [f"19{i}0.gz" for i in range(0,10)] 
colls =["economist2", "forbes", "bloomberg", "fortune", "hbr"]
filnames = ["first", "second", "third"]

#filnames = [f"first_spirit_{coll}" for coll in colls]
losses = {filname: {} for filname in filnames}
for filname in filnames:
    logging.basicConfig(filename=f'{output_dir}/{filname}.log', encoding='utf-8', level=logging.DEBUG)
    log_fil = open(f'{output_dir}/{filname}_log.txt', mode = 'w')
    #ngrams = ngram_extractor_lite(filname=filname)#, blank=infos["blank_ind"][filname])
    txts = [f"{filname}_spirit_{coll}.txt" for coll in colls]
    ngrams = []
    for txt in txts:
        content = file2sent(txt)
        ngrams += content
        print(ngrams[:10])
        del content
    print("Starting training...")
    model = gensim.models.word2vec.Word2Vec(ngrams,sg=1, vector_size=300, window=5, min_count=5, compute_loss=True, callbacks = [callback()], workers=10, hs=0, negative=8) 
    #default epochs = 5, size = vector dimensions, min_count= min count for word to be considered, workers for multiprocessing, sg=1 = skipgram (0 = CBOW), hs=1 = softmax used, negative = no. noise words used in negative sampling
    model.save(output_dir + "w2vmodel_ng5_sg_" + filname +'_full')
    pd.DataFrame(losses).to_csv(f"{output_dir}/{filname}_losses.csv")


In [7]:
os.chdir("D:/google_ngrams/processed_data/us_12_final_burst")
output_dir = "D:/google_ngrams/Vectors/us_12_final/"

filnames = [f"19{i}0.gz" for i in range(0,10)] 
#colls =["economist2", "forbes", "bloomberg", "fortune", "hbr"]
#filnames = ["first", "second", "third"]

#filnames = [f"first_spirit_{coll}" for coll in colls]
losses = {filname: {} for filname in filnames}
for filname in filnames[::-1]:
   # logging.basicConfig(filename=f'{output_dir}/{filname}.log', encoding='utf-8', level=logging.DEBUG)
    log_fil = open(f'{output_dir}/{filname}_log.txt', mode = 'w')
    ngrams = ngram_extractor_lite(filname=filname)#, blank=infos["blank_ind"][filname])
    #txts = [f"{filname}_spirit_{coll}.txt" for coll in colls]
    #ngrams = []
    #for txt in txts:
    #    content = file2sent(txt)
    #    ngrams += content
    #    print(ngrams[:10])
    #    del content
    print("Starting training...")
    model = gensim.models.word2vec.Word2Vec(ngrams,sg=1, vector_size=300, window=5, min_count=5, compute_loss=True, callbacks = [callback()], workers=10, hs=0, negative=8) 
    #default epochs = 5, size = vector dimensions, min_count= min count for word to be considered, workers for multiprocessing, sg=1 = skipgram (0 = CBOW), hs=1 = softmax used, negative = no. noise words used in negative sampling
    model.save(output_dir + "w2vmodel_ng5_sg_" + filname +'_full')
    pd.DataFrame(losses).to_csv(f"{output_dir}/{filname}_losses.csv")


Starting training...
n 1990.gz


1126000it [00:07, 155381.26it/s]


error: Error -3 while decompressing data: invalid block type